# Microsoft GraphRAG on Databricks (FMAPI版)

このノートブックでは、MicrosoftのGraphRAGをDatabricks上で実行する方法を説明します。

## GraphRAGとは

GraphRAGは、ナレッジグラフを活用したRAG(Retrieval-Augmented Generation)手法です。
従来のベクトル検索ベースのRAGと異なり、以下の特徴があります:

- **エンティティとリレーションシップの抽出**: テキストから人物、組織、技術などのエンティティと、それらの関係性を抽出
- **コミュニティ検出**: 関連するエンティティをクラスタリングし、階層的なコミュニティ構造を構築
- **Global Search**: コミュニティレポートを活用し、データセット全体に関する高レベルな質問に回答
- **Local Search**: 特定のエンティティに関する詳細な質問に回答

## 前提条件

- Databricks Runtime 14.0 ML以上
- **クラスター**(サーバレスではなく)を使用(LanceDBのファイル操作制約のため)
- FMAPIでOpenAIモデル(databricks-gpt-5-2等)が利用可能であること
- Unity Catalogが有効なワークスペース

## 参考リンク

- [GraphRAG GitHub](https://github.com/microsoft/graphrag)
- [GraphRAG Documentation](https://microsoft.github.io/graphrag/)

## 1. パッケージのインストール

In [0]:
%pip install graphrag pyvis --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.12.1 requires azure-cosmos==4.3.1, but you have azure-cosmos 4.14.3 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

## 2. 作業ディレクトリの設定

GraphRAGはLanceDBを使用してベクトルデータを保存しますが、LanceDBの`rename`操作は
Unity CatalogボリュームやWorkspaceファイルシステムでサポートされていません。
そのため、**ローカルディスク(`/tmp`)** を作業ディレクトリとして使用します。

処理完了後、結果はDelta Tableに保存して永続化します。

In [0]:
import os
import shutil

# Unity Catalog設定
CATALOG = "takaakiyayoi_catalog"
SCHEMA = "default"

# 作業ディレクトリ(クラスターのローカルディスク)
WORK_DIR = "/tmp/graphrag_work"
INPUT_DIR = f"{WORK_DIR}/input"
OUTPUT_DIR = f"{WORK_DIR}/output"
CACHE_DIR = f"{WORK_DIR}/cache"

# 既存のディレクトリがあればクリア
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# ディレクトリ作成
for d in [WORK_DIR, INPUT_DIR, OUTPUT_DIR, CACHE_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"作業ディレクトリ: {WORK_DIR}")

作業ディレクトリ: /tmp/graphrag_work


## 3. サンプルデータの準備

GraphRAGのインデックス作成には、テキストデータが必要です。
ここではDatabricksエコシステムに関するサンプルテキストを使用します。

実際の利用では、このディレクトリに分析したいテキストファイル(.txt)を配置してください。

In [0]:
sample_text = """
Databricksは、データエンジニアリング、データサイエンス、機械学習のための統合プラットフォームです。
Apache Sparkの創設者たちによって設立され、レイクハウスアーキテクチャを提唱しています。

Unity Catalogは、Databricksのデータガバナンスソリューションです。
データ、MLモデル、AIアセットを一元管理し、きめ細かなアクセス制御を提供します。
Unity CatalogはDatabricksワークスペース全体でメタデータを共有できます。

MLflowは、機械学習のライフサイクル管理のためのオープンソースプラットフォームです。
実験追跡、モデル登録、デプロイメントなどの機能を提供します。
MLflow 3では、LoggedModelという新しい概念が導入され、GenAIアプリケーションの管理が強化されました。

Delta Lakeは、データレイクに信頼性をもたらすオープンソースのストレージレイヤーです。
ACIDトランザクション、スキーマエンフォースメント、タイムトラベルなどの機能を提供します。
DatabricksではDelta Lakeがデフォルトのテーブル形式として使用されています。

Mosaic AIは、Databricksの生成AIソリューションです。
Foundation Modelのファインチューニング、RAGアプリケーション構築、AIエージェント開発をサポートします。
Agent Frameworkを使用すると、本番環境向けのAIエージェントを構築できます。
"""

with open(f"{INPUT_DIR}/databricks_overview.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

print(f"サンプルデータを保存しました: {INPUT_DIR}/databricks_overview.txt")

サンプルデータを保存しました: /tmp/graphrag_work/input/databricks_overview.txt


## 4. FMAPI設定

Databricks Foundation Model API(FMAPI)で提供されているOpenAIモデルを使用します。

**注意**: FMAPIのLlamaモデルはJSON mode制約によりGraphRAGと互換性がありません。
OpenAIモデル(databricks-gpt-5-2等)を使用してください。

In [0]:
# Databricks認証情報
DATABRICKS_HOST = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# FMAPI設定
LLM_API_KEY = DATABRICKS_TOKEN
LLM_API_BASE = f"{DATABRICKS_HOST}/serving-endpoints"

# FMAPIで利用可能なOpenAIモデルを使用
LLM_MODEL = "databricks-gpt-5-2"
EMBED_MODEL = "databricks-gte-large-en"

print(f"Databricks Host: {DATABRICKS_HOST}")
print(f"LLM Model: {LLM_MODEL}")
print(f"Embedding Model: {EMBED_MODEL}")

Databricks Host: https://tokyo.cloud.databricks.com
LLM Model: databricks-gpt-5-2
Embedding Model: databricks-gte-large-en


## 5. 設定ファイルの作成

GraphRAGの動作は`settings.yaml`で制御します。

**FMAPIでのポイント:**
- `model_supports_json: true`を指定
- `max_tokens`を明示的に指定(FMAPIは`null`を受け付けない)

In [0]:
settings_content = f"""encoding_model: cl100k_base

models:
  default_chat_model:
    type: openai_chat
    api_key: {LLM_API_KEY}
    api_base: {LLM_API_BASE}
    model: {LLM_MODEL}
    encoding_model: cl100k_base
    model_supports_json: true
    max_tokens: 4096
    max_retries: 3
    request_timeout: 180

  default_embedding_model:
    type: openai_embedding
    api_key: {LLM_API_KEY}
    api_base: {LLM_API_BASE}
    model: {EMBED_MODEL}
    encoding_model: cl100k_base

chunks:
  size: 1200
  overlap: 100
  group_by_columns:
    - id

input:
  type: file
  file_type: text
  base_dir: "{INPUT_DIR}"
  file_encoding: utf-8
  file_pattern: ".*\\\\.txt$$"

cache:
  type: file
  base_dir: "{CACHE_DIR}"

storage:
  type: file
  base_dir: "{OUTPUT_DIR}"

reporting:
  type: file
  base_dir: "{OUTPUT_DIR}"

entity_extraction:
  entity_types:
    - organization
    - person
    - technology
    - concept
  max_gleanings: 1

summarize_descriptions:
  max_length: 500

claim_extraction:
  enabled: false

community_reports:
  max_length: 2000
  max_input_length: 8000

cluster_graph:
  max_cluster_size: 10

embed_graph:
  enabled: false

umap:
  enabled: false

snapshots:
  graphml: true
  raw_entities: true
  top_level_nodes: true
"""

with open(f"{WORK_DIR}/settings.yaml", "w", encoding="utf-8") as f:
    f.write(settings_content)

print(f"設定ファイルを保存しました: {WORK_DIR}/settings.yaml")

設定ファイルを保存しました: /tmp/graphrag_work/settings.yaml


## 6. インデックスの作成

GraphRAGのインデックス作成プロセスでは、以下の処理が行われます:

1. テキストをチャンクに分割
2. LLMでエンティティとリレーションシップを抽出
3. ナレッジグラフを構築
4. コミュニティ検出(Leidenアルゴリズム)
5. 各コミュニティのサマリーレポート生成
6. エンティティ説明のEmbedding生成

このプロセスにはLLMへの複数回のAPI呼び出しが発生するため、数分かかる場合があります。

In [0]:
os.chdir(WORK_DIR)
!graphrag index --root .

2026-01-11 23:08:45.568388: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-11 23:08:45.569228: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 23:08:45.572400: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 23:08:45.581585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768172925.598047    3646 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768172925.60

DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-9 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Thr

## 7. 作成されたデータの確認

インデックス作成により、以下のParquetファイルが生成されます:

- `entities.parquet`: 抽出されたエンティティ
- `relationships.parquet`: エンティティ間の関係性
- `communities.parquet`: コミュニティ構造
- `community_reports.parquet`: 各コミュニティのサマリー
- `text_units.parquet`: 分割されたテキストチャンク

In [0]:
import pandas as pd

print("=== 出力ファイル ===")
for f in sorted(os.listdir(OUTPUT_DIR)):
    file_path = f"{OUTPUT_DIR}/{f}"
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path) / 1024
        print(f"  {f}: {size:.1f} KB")

=== 出力ファイル ===
  communities.parquet: 10.9 KB
  community_reports.parquet: 57.6 KB
  context.json: 0.0 KB
  documents.parquet: 13.2 KB
  entities.parquet: 13.2 KB
  graph.graphml: 4.9 KB
  indexing-engine.log: 48.7 KB
  relationships.parquet: 10.3 KB
  stats.json: 1.0 KB
  text_units.parquet: 17.1 KB


### エンティティの確認

抽出されたエンティティ(人物、組織、技術、概念など)を確認します。

In [0]:
entities_df = pd.read_parquet(f"{OUTPUT_DIR}/entities.parquet")
print(f"エンティティ数: {len(entities_df)} 件")
display(entities_df[["title", "type", "description"]])

エンティティ数: 37 件


title,type,description
DATABRICKS,ORGANIZATION,"Databricks is an integrated platform for data engineering, data science, and machine learning. It was founded by the creators of Apache Spark and promotes the Lakehouse architecture. Within Databricks, products such as Unity Catalog, Delta Lake (as the default table format), and Mosaic AI are positioned as key capabilities/solutions.>"
APACHE SPARK,ORGANIZATION,Apache Spark is an open-source data processing engine whose creators founded Databricks. It is referenced as the originating technology/community behind Databricks’ founding.>
LAKEHOUSE ARCHITECTURE,EVENT,"Lakehouse architecture is an architectural approach advocated by Databricks, positioned as a unifying paradigm for data engineering/analytics and machine learning workloads on a shared data foundation.>"
UNITY CATALOG,ORGANIZATION,"Unity Catalog is Databricks’ data governance solution. It centrally manages data, ML models, and AI assets, provides fine-grained access control, and enables metadata sharing across Databricks workspaces.>"
MLFLOW,ORGANIZATION,"MLflow is an open-source platform for managing the machine learning lifecycle, providing capabilities such as experiment tracking, model registry, and deployment. It is described as a platform with versioned evolution including MLflow 3.>"
MLFLOW 3,EVENT,"MLflow 3 is a major version release of MLflow in which a new concept called LoggedModel was introduced, strengthening management of GenAI applications.>"
LOGGEDMODEL,EVENT,LoggedModel is a new concept introduced in MLflow 3 intended to enhance management of GenAI applications within the MLflow lifecycle management framework.>
DELTA LAKE,ORGANIZATION,"Delta Lake is an open-source storage layer for data lakes that adds reliability features such as ACID transactions, schema enforcement, and time travel. In Databricks, Delta Lake is used as the default table format.>"
MOSAIC AI,ORGANIZATION,"Mosaic AI is Databricks’ generative AI solution. It supports foundation model fine-tuning, building RAG applications, and developing AI agents, including production-oriented agent development via an Agent Framework.>"
AGENT FRAMEWORK,ORGANIZATION,Agent Framework is a framework referenced under Mosaic AI that enables building production-ready AI agents.>


### リレーションシップの確認

エンティティ間の関係性を確認します。

In [0]:
relationships_df = pd.read_parquet(f"{OUTPUT_DIR}/relationships.parquet")
print(f"リレーションシップ数: {len(relationships_df)} 件")
display(relationships_df[["source", "target", "description"]])

リレーションシップ数: 35 件


source,target,description
DATABRICKS,APACHE SPARK,"Databricks was founded by the creators of Apache Spark, linking the company/platform’s origin to the Spark project and its founders."
DATABRICKS,LAKEHOUSE ARCHITECTURE,Databricks advocates (promotes) the Lakehouse architecture as a core architectural approach associated with its platform.
DATABRICKS,UNITY CATALOG,Unity Catalog is described as Databricks’ data governance solution and operates across Databricks workspaces to share metadata and enforce access control.
DATABRICKS,DELTA LAKE,"Delta Lake is used within Databricks as the default table format, indicating a strong product/platform integration relationship."
DATABRICKS,MOSAIC AI,"Mosaic AI is described as Databricks’ generative AI solution, making it a first-party solution within the Databricks platform ecosystem."
DATABRICKS,DATA ENGINEERING,Databricks is described as an integrated platform for data engineering workloads.
DATABRICKS,DATA SCIENCE,Databricks is described as an integrated platform for data science workloads.
DATABRICKS,MACHINE LEARNING,Databricks is described as an integrated platform for machine learning workloads.
UNITY CATALOG,DATA GOVERNANCE,Unity Catalog is explicitly described as Databricks’ data governance solution.
UNITY CATALOG,ACCESS CONTROL,Unity Catalog provides fine-grained access control over governed assets.


### コミュニティレポートの確認

関連するエンティティのクラスタ(コミュニティ)と、そのサマリーを確認します。

In [0]:
reports_df = pd.read_parquet(f"{OUTPUT_DIR}/community_reports.parquet")
print(f"コミュニティレポート数: {len(reports_df)} 件\n")

for _, row in reports_df.iterrows():
    print(f"=== {row['title']} ===")
    print(f"{row['summary']}\n")

コミュニティレポート数: 5 件

=== Databricks Unity Catalog Governance: Access Control, Metadata, and Governed AI/ML Assets ===
This community centers on Unity Catalog, described as Databricks’ data governance solution, and its role as a hub that connects governance functions to multiple governed asset types and environments. Unity Catalog is directly linked to data governance, fine-grained access control, metadata sharing, and operation across Databricks workspaces, indicating a centralized control plane for policy and visibility. It also centrally manages ML models and broader AI assets as governed assets, suggesting the community’s focus is on consistent governance and control across both data and AI/ML artifacts within the Databricks platform. [Data: Entities (3, 12, 15, 14, 13); Relationships (8, 9, 10, 11, 12, +more)]

=== Delta Lake Reliability Features for Data Lakes (ACID, Schema Enforcement, Time Travel) ===
This community centers on Delta Lake, an open-source storage layer positioned as 

## 8. ナレッジグラフの可視化

抽出されたエンティティとリレーションシップをグラフとして可視化します。
PyVisを使用してインタラクティブなネットワーク図を生成します。

In [0]:
from pyvis.network import Network
import networkx as nx

# NetworkXグラフを作成
G = nx.Graph()

# エンティティをノードとして追加
for _, row in entities_df.iterrows():
    G.add_node(
        row["title"],
        title=row["description"][:200] if pd.notna(row["description"]) else "",
        group=row["type"] if pd.notna(row.get("type")) else "unknown"
    )

# リレーションシップをエッジとして追加
for _, row in relationships_df.iterrows():
    if row["source"] in G.nodes and row["target"] in G.nodes:
        G.add_edge(
            row["source"],
            row["target"],
            title=row["description"] if pd.notna(row["description"]) else ""
        )

print(f"グラフ統計: {G.number_of_nodes()} ノード, {G.number_of_edges()} エッジ")

グラフ統計: 37 ノード, 35 エッジ


In [0]:
# PyVisでインタラクティブな可視化
net = Network(height="600px", width="100%", bgcolor="#ffffff", font_color="black", notebook=True)
net.from_nx(G)
net.toggle_physics(True)
net.set_options("""
{
  "nodes": {
    "font": {"size": 14},
    "scaling": {"min": 10, "max": 30}
  },
  "edges": {
    "color": {"inherit": true},
    "smooth": {"type": "continuous"}
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.01,
      "springLength": 100
    },
    "solver": "forceAtlas2Based"
  }
}
""")

graph_html_path = f"{OUTPUT_DIR}/knowledge_graph.html"
net.save_graph(graph_html_path)
print(f"グラフを保存しました: {graph_html_path}")

グラフを保存しました: /tmp/graphrag_work/output/knowledge_graph.html


### グラフの表示

以下のセルでインタラクティブなナレッジグラフを表示します。
ノードをドラッグして移動したり、ホバーして詳細を確認できます。

In [0]:
with open(graph_html_path, "r", encoding="utf-8") as f:
    html_content = f.read()

displayHTML(html_content)

<!-- 
-->

## 9. Global Search

Global Searchは、コミュニティレポートを活用して、データセット全体に関する高レベルな質問に回答します。
「このデータセットの主要なテーマは何か?」といった要約的な質問に適しています。

In [0]:
from pathlib import Path
import graphrag.api as api
from graphrag.config.load_config import load_config

graphrag_config = load_config(Path(WORK_DIR))

entities = pd.read_parquet(f"{OUTPUT_DIR}/entities.parquet")
communities = pd.read_parquet(f"{OUTPUT_DIR}/communities.parquet")
community_reports = pd.read_parquet(f"{OUTPUT_DIR}/community_reports.parquet")

2026-01-11 23:10:47.203378: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-11 23:10:47.204210: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 23:10:47.207612: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-11 23:10:47.217061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768173047.233259    3562 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768173047.23

In [0]:
query = "このデータセットで説明されている主要な技術とその関係性を要約してください"

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query=query,
)

print("=== 質問 ===")
print(query)
print("\n=== 回答 ===")
print(response)

=== 質問 ===
このデータセットで説明されている主要な技術とその関係性を要約してください

=== 回答 ===
## 全体像（主要技術の配置）
このデータセットでは、**Databricks**が統合プラットフォームとして中心に位置づけられ、**データエンジニアリング／データサイエンス／機械学習**の各ワークロードを単一基盤で扱う構造が示されています。また、Apache Sparkの創設者によって設立されたという起源的関係が示され、さらに**Lakehouseアーキテクチャ**を推進することで、分析とMLを共有データ基盤上で統合する思想が関係性として説明されています [Data: Reports (4)]。

## データ基盤：Delta Lake（ストレージ層）と信頼性機能
**Delta Lake**は、データレイク向けのオープンソース・ストレージレイヤとして中心にあり、信頼性機能として **ACIDトランザクション／スキーマ強制（schema enforcement）／タイムトラベル（過去バージョンの参照・復元）** と直接結びついています。加えて、Databricksにおける**デフォルトのテーブルフォーマット**として利用される関係が示され、プラットフォーム標準のデータ表現として位置づけられています [Data: Reports (1)]。

## ガバナンス：Unity Catalog（統制のハブ）
**Unity Catalog**はDatabricksのデータガバナンスソリューションとしてハブになっており、**データガバナンス／きめ細かなアクセス制御／メタデータ共有／複数Databricksワークスペース横断の運用**に関係づけられています。さらに、MLモデルやより広いAI資産（例：**モデル、プロンプト、エージェント、アプリケーション**）も「ガバナンス対象資産」として中央管理することで、**データとAI/ML成果物を同一の統制平面で扱う**関係性が示されています [Data: Reports (0)]。

## 生成AI：Mosaic AI（生成AI機能のハブ）
**Mosaic AI**はDatabricksの生成AIソリューションとしてハブになっており、生成AI領域の主要機能として、(1) **基盤モデル（Foundation Mod

[Trace(trace_id=tr-d245d38d4dbf1a1ea1affcdfbd8c88f7), Trace(trace_id=tr-dd20b269e21a194a1a45679e60e0025d)]

## 10. Local Search

Local Searchは、特定のエンティティに関する詳細な質問に回答します。
ベクトル検索でエンティティを特定し、関連するコンテキストを組み合わせて回答を生成します。

In [0]:
text_units = pd.read_parquet(f"{OUTPUT_DIR}/text_units.parquet")
relationships = pd.read_parquet(f"{OUTPUT_DIR}/relationships.parquet")

query_local = "MLflowとは何ですか?どのような機能がありますか?"

response_local, context_local = await api.local_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    text_units=text_units,
    relationships=relationships,
    covariates=None,
    community_level=2,
    response_type="Multiple Paragraphs",
    query=query_local,
)

print("=== 質問 ===")
print(query_local)
print("\n=== 回答 ===")
print(response_local)

=== 質問 ===
MLflowとは何ですか?どのような機能がありますか?

=== 回答 ===
## MLflowとは何か

MLflowは、機械学習（ML）のライフサイクル管理のためのオープンソースプラットフォームです。具体的には、モデル開発から運用（デプロイ）までの一連の流れを管理するための仕組みを提供します [Data: Sources (0)]。

## どのような機能があるか

提供されている情報の範囲では、MLflowの主な機能として以下が挙げられます。

- **実験追跡（Experiment Tracking）**：学習実験の結果やパラメータなどを追跡・記録する機能 [Data: Sources (0)]。  
- **モデル登録（Model Registry）**：学習したモデルを登録し、管理する機能 [Data: Sources (0)]。  
- **デプロイメント（Deployment）**：登録・管理したモデルを実運用環境へ展開するための機能 [Data: Sources (0)]。

## MLflow 3での追加要素（提供データに基づく）

MLflow 3では、**LoggedModel**という新しい概念が導入され、**生成AI（GenAI）アプリケーションの管理が強化**されたとされています [Data: Sources (0)]。

## 補足（このデータで分からないこと）

このデータには、MLflowの各機能の具体的な操作方法、対応する実行環境、他のDatabricks製品（例：Unity Catalog）との詳細な連携手順までは記載がありません。そのため、ここでは上記の範囲を超える断定はできません。


[Trace(trace_id=tr-c26dcc58c47db24794b4a463001486fc), Trace(trace_id=tr-e29c0716c876bc2c0fecb7cc5dc33d19)]

## 11. Delta Tableへの保存

ローカルディスク上のデータはクラスター終了時に消えてしまうため、
Delta Tableに保存して永続化します。

次回以降は、インデックス作成をスキップしてDelta Tableからデータを読み込んでクエリを実行できます。

In [0]:
tables = {
    "graphrag_entities": "entities.parquet",
    "graphrag_relationships": "relationships.parquet",
    "graphrag_community_reports": "community_reports.parquet",
    "graphrag_communities": "communities.parquet",
    "graphrag_text_units": "text_units.parquet",
}

for table_name, file_name in tables.items():
    file_path = f"file:{OUTPUT_DIR}/{file_name}"
    spark.read.parquet(file_path).write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.{table_name}")
    print(f"✓ {CATALOG}.{SCHEMA}.{table_name}")

✓ takaakiyayoi_catalog.default.graphrag_entities
✓ takaakiyayoi_catalog.default.graphrag_relationships
✓ takaakiyayoi_catalog.default.graphrag_community_reports
✓ takaakiyayoi_catalog.default.graphrag_communities
✓ takaakiyayoi_catalog.default.graphrag_text_units


## 12. (オプション) Delta Tableからの読み込み

保存したDelta Tableからデータを読み込んでクエリを実行する例です。

In [0]:
# # Delta Tableからデータを読み込む場合
# entities = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_entities").toPandas()
# communities = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_communities").toPandas()
# community_reports = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_community_reports").toPandas()
# text_units = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_text_units").toPandas()
# relationships = spark.read.table(f"{CATALOG}.{SCHEMA}.graphrag_relationships").toPandas()

## まとめ

このノートブックでは、以下を実行しました:

1. **インデックス作成**: テキストからエンティティ、リレーションシップ、コミュニティを抽出
2. **可視化**: ナレッジグラフをインタラクティブに表示
3. **Global Search**: データセット全体に関する質問に回答
4. **Local Search**: 特定のエンティティに関する質問に回答
5. **永続化**: 結果をDelta Tableに保存

### FMAPIでの注意点

| 項目 | 設定 |
|------|------|
| モデル | OpenAIモデル(databricks-gpt-5-2等)を使用 |
| max_tokens | 明示的に指定(FMAPIはnullを受け付けない) |
| model_supports_json | true |

**注意**: FMAPIのLlamaモデルはJSON mode制約によりGraphRAGのクエリと互換性がありません。

### その他の制約

- **ストレージ**: LanceDBの制約により、クラスターのローカルディスクを使用
- **コスト**: インデックス作成時にLLMへの多数のAPI呼び出しが発生